In [9]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd

# 과제. 네이버 증권 크롤링

* LG 전자 일간 데이터를 재귀호출을 통해 크롤링해서 가져온 다음, csv 파일로 저장한다.

In [13]:
# 크롤링 할 사이트 : LG전자 일별 데이터, base url 설정
sise_url = "https://finance.naver.com/item/sise_day.nhn?code=066570&page="

In [15]:
# html 로 읽기 & 확인
table = pd.read_html(f"{sise_url}1")
table

[            날짜       종가     전일비       시가       고가       저가        거래량
 0          NaN      NaN     NaN      NaN      NaN      NaN        NaN
 1   2020.08.11  83000.0  2700.0  86300.0  86300.0  83000.0  1678036.0
 2   2020.08.10  85700.0  8500.0  77100.0  86200.0  76500.0  3706582.0
 3   2020.08.07  77200.0     0.0  77800.0  78300.0  76300.0   759800.0
 4   2020.08.06  77200.0  1100.0  78300.0  78900.0  76900.0   899941.0
 5   2020.08.05  78300.0   400.0  78900.0  78900.0  76500.0  1197933.0
 6          NaN      NaN     NaN      NaN      NaN      NaN        NaN
 7          NaN      NaN     NaN      NaN      NaN      NaN        NaN
 8          NaN      NaN     NaN      NaN      NaN      NaN        NaN
 9   2020.08.04  77900.0  3900.0  74900.0  78100.0  74000.0  1738014.0
 10  2020.08.03  74000.0  3300.0  71300.0  74500.0  71000.0  1183563.0
 11  2020.07.31  70700.0  1400.0  72200.0  72500.0  70600.0   851076.0
 12  2020.07.30  72100.0   100.0  72700.0  73500.0  71800.0   735701.0
 13  2

In [16]:
response_sise = requests.get(f'{sise_url}1')

In [17]:
type(response_sise.text)

str

In [18]:
html_sise = bs(response_sise.text)

In [19]:
type(html_sise)

bs4.BeautifulSoup

In [20]:
html_sise

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="/css/newstock.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/newstock2.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/newstock3.css?20200806200958" rel="stylesheet" type="text/css"/>
<link href="/css/world.css?20200806200958" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="JavaScript">
function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";
}
</script>
<h4 class="tlline2"><strong><span class="red03">일별</span>시세</strong></h4>
<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th>

In [21]:
# tr 개수 확인 
trs = html_sise.select('body > table.type2> tr')
len(trs)

16

In [22]:
# 0 번째는 칼럼명 => 크롤링 시 제외
trs[0]

<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>

In [23]:
# 2번째 tr은 일간시세 테이블의 첫번쨰 행임 
trs[2]

<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2020.08.11</span></td>
<td class="num"><span class="tah p11">83,000</span></td>
<td class="num">
<img alt="하락" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_down.gif" style="margin-right:4px;" width="7"/><span class="tah p11 nv01">
				2,700
				</span>
</td>
<td class="num"><span class="tah p11">86,300</span></td>
<td class="num"><span class="tah p11">86,300</span></td>
<td class="num"><span class="tah p11">83,000</span></td>
<td class="num"><span class="tah p11">1,678,036</span></td>
</tr>

In [24]:
# page.1 의 1st row 날짜 
trs[2].select("td")[0].get_text(strip=True)

'2020.08.11'

In [25]:
# page.1 의 1st row 종가 
trs[2].select("td")[1].get_text(strip=True)

'83,000'

In [26]:
# page.1 의 1st row 전일비 대비 상태 
trs[2].select("img")[0]['alt']

'하락'

In [34]:
# page.1 의 1st row 전일비 
# 2008 년 10/30 데이터에서 전일비가 10,000 이상 차이가 나면 'gif'링크가 삽입되어,
#'alt' 가 잡히지 않는 키에러가 발생하여 'elif' 문 추후 추가 

if trs[3].select('img') == []:
    trs[3].select("td")[2].get_text(strip=True)
elif trs[6].select('img')[0]['src'] == 'https://ssl.pstatic.net/imgstock/images/images4/ico_up02.gif':
        전일비 = '+' + tr.select("td")[2].get_text(strip=True)
elif trs[13].select('img')[0]['src'] == 'https://ssl.pstatic.net/imgstock/images/images4/ico_down02.gif':
        전일비 = '-' + tr.select("td")[2].get_text(strip=True)
else:
    ('+' if trs[2].select("img")[0]['alt'] else '-') + trs[2].select("td")[1].get_text(strip=True)

In [29]:
# page.1 의 1st row '시가'
trs[2].select("td")[3].get_text(strip=True)

'86,300'

In [30]:
# page.1 의 1st row '고가'
trs[2].select("td")[4].get_text(strip=True)

'86,300'

In [31]:
# page.1 의 1st row '저가'
trs[2].select("td")[5].get_text(strip=True)

'83,000'

In [32]:
# page.1 의 1st row '거래량'
trs[2].select("td")[-1].get_text(strip=True)

'1,678,036'

In [33]:
def parse_tr(tr) :
    info = []
    
    날짜 = tr.select("td")[0].get_text(strip=True)
    종가 = tr.select("td")[1].get_text(strip=True)
    if tr.select('img') == []:
        전일비 = tr.select("td")[2].get_text(strip=True)
    elif tr.select('img')[0]['src'] == 'https://ssl.pstatic.net/imgstock/images/images4/ico_up02.gif':
        전일비 = '+' + tr.select("td")[2].get_text(strip=True)
    elif tr.select('img')[0]['src'] == 'https://ssl.pstatic.net/imgstock/images/images4/ico_down02.gif':
        전일비 = '-' + tr.select("td")[2].get_text(strip=True)
    else:
        전일비 = ('+' if tr.select("img")[0]['alt'] else '-') + tr.select("td")[2].get_text(strip=True)
    시가 = tr.select("td")[3].get_text(strip=True)
    고가 = tr.select("td")[4].get_text(strip=True)
    저가 = tr.select("td")[5].get_text(strip=True)
    거래량 = tr.select("td")[-1].get_text(strip=True)
    
    info.append(날짜)
    info.append(종가)
    info.append(전일비)
    info.append(시가)
    info.append(고가)
    info.append(저가)
    info.append(거래량)
    
    return info
    

In [319]:
# 중간 재귀호출 점검 시, 키에러가 발생한 부분 체크하여, 해당 특정 gif 링크 가져오기
sise_url1 = "https://finance.naver.com/item/sise_day.nhn?code=066570&page=292"
html1 = bs(requests.get(sise_url1).text)
str1 = html1.select('body > table.type2 > tr')
str1[6].select('img')[0]['src']

'https://ssl.pstatic.net/imgstock/images/images4/ico_up02.gif'

In [325]:
# elif 추가 후, 오류났던 10.24 데이터 제대로 출력되는지 확인 
parse_tr(str1[13])

['2008.10.24', '70,600', '-12,400', '82,200', '83,000', '70,600', '2,222,392']

In [209]:
# 확인
parse_tr(trs[2])

['2020.08.10', '85,700', '+8,500', '77,100', '86,200', '76,500', '3,683,348']

In [35]:
# 한 페이지의 모든 각 행을 리스트로 묶고, 하나의 리스트로 출력되도록 함수 입력 
# page2 는 extend 가 append 와 다른 점 알려고 만든 것 
# trs[0, 1, 7, 8, 9, 15] 는 시가 데이터가 아니므로 제외 

page = []
# page2 = []
for i in range(0, len(trs)):
    if i in [0, 1, 7, 8, 9, 15]: 
        pass
    else:
        page.append(parse_tr(trs[i]))
#         page2.extend(parse_tr(trs[i]))

print(page)
# page2

[['2020.08.11', '83,000', '+2,700', '86,300', '86,300', '83,000', '1,678,036'], ['2020.08.10', '85,700', '+8,500', '77,100', '86,200', '76,500', '3,706,582'], ['2020.08.07', '77,200', '0', '77,800', '78,300', '76,300', '759,800'], ['2020.08.06', '77,200', '+1,100', '78,300', '78,900', '76,900', '899,941'], ['2020.08.05', '78,300', '+400', '78,900', '78,900', '76,500', '1,197,933'], ['2020.08.04', '77,900', '+3,900', '74,900', '78,100', '74,000', '1,738,014'], ['2020.08.03', '74,000', '+3,300', '71,300', '74,500', '71,000', '1,183,563'], ['2020.07.31', '70,700', '+1,400', '72,200', '72,500', '70,600', '851,076'], ['2020.07.30', '72,100', '+100', '72,700', '73,500', '71,800', '735,701'], ['2020.07.29', '72,000', '+500', '72,500', '73,400', '71,700', '804,210']]


In [36]:
# 전체 LG 전자 일간 시세 정보를 담아줄 비어있는 리스트를 만듭니다.
import numpy as np
random_time = np.random.uniform(0, 1)
random_time

def crawling_daily_sise_LG(pnum):
    # random_time 만큼 쉬었다가 크롤링합니다.
#     time.sleep(random_time)

    url = f"{sise_url}{pnum}"
    response = requests.get(url)
    html = bs(response.text)
    last_page = int(html_sise.select('body > table.Nnavi > tr > td.pgRR')[0].select('a')[0]['href'].split('=')[-1])

    if pnum <= last_page and pnum > 0 :
        trs = html.select('body > table.type2 > tr')
        return trs
    else:
        return False

In [37]:
# 함수 작동 확인 
crawling_daily_sise_LG(0)

False

In [49]:
# 재귀호출을 사용하는 방법
all_daily_sise_LG = []

def crawling_all_daily_sise_LG(pnum):
    trs = crawling_daily_sise_LG (pnum)
    if trs :
        page = []
        for i in range(0, len(trs)):
            if i in [0, 1, 7, 8, 9, 15]:
                pass
            else:
                page.append(parse_tr(trs[i]))
        
        all_daily_sise_LG.extend(page)
        # 결과가 정상동작한다면 다음 페이지 번호를 호출합니다.
        pnum = pnum + 1
        # 재귀호출 => 자기 자신을 호출합니다.
        return crawling_all_daily_sise_LG(pnum)
    else:
        # 페이지가 없다면 all_station 을 반환합니다.
        return all_daily_sise_LG

In [51]:
# 전체페이지 크롤링하기 
df = crawling_all_daily_sise_LG(1)

In [52]:
header = ['날짜', '종가', '전일비','시가', '고가', '저가', '거래량']
all_daily_LG = pd.DataFrame(df, columns = header)
all_daily_LG

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.08.11,"83,000","+2,700","86,300","86,300","83,000","1,678,036"
1,2020.08.10,"85,700","+8,500","77,100","86,200","76,500","3,706,582"
2,2020.08.07,"77,200",0,"77,800","78,300","76,300","759,800"
3,2020.08.06,"77,200","+1,100","78,300","78,900","76,900","899,941"
4,2020.08.05,"78,300",+400,"78,900","78,900","76,500","1,197,933"
...,...,...,...,...,...,...,...
9055,2002.04.25,"52,400","+4,600","55,000","56,400","51,800","3,913,730"
9056,2002.04.24,"57,000","+1,800","59,500","60,800","54,300","4,659,371"
9057,2002.04.23,"58,800","+5,600","62,100","62,800","58,500","5,384,435"
9058,2002.04.22,"64,400","+7,600","56,800","65,300","56,800","8,151,792"


In [54]:
all_daily_LG.to_csv("all_daily_LG.csv", index = False)

In [55]:
all_daily_LG = pd.read_csv("all_daily_LG.csv")

In [56]:
all_daily_LG.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.08.11,"83,000","+2,700","86,300","86,300","83,000","1,678,036"
1,2020.08.10,"85,700","+8,500","77,100","86,200","76,500","3,706,582"
2,2020.08.07,"77,200",0,"77,800","78,300","76,300","759,800"
3,2020.08.06,"77,200","+1,100","78,300","78,900","76,900","899,941"
4,2020.08.05,"78,300",+400,"78,900","78,900","76,500","1,197,933"
